In [1]:
import pandas as pd
pd.set_option('display.max_rows', 300)  # Shows up to 50 rows

In [2]:
vertices = pd.read_csv("../../preds1000/strainselect/StrainSelect21_vertices.tab.txt",sep="\t")

/scratch/slurm_tmpdir/job_23519316/ipykernel_575003/833439689.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  vertices = pd.read_csv("../../preds1000/strainselect/StrainSelect21_vertices.tab.txt",sep="\t")


In [3]:
vertices.vertex_type.value_counts()

vertex_type
g16               884828
gss               878311
contig            720657
gb_assembly       275232
rs_assembly       259195
wgs_master_pre    245975
patric_genome     234506
gold_org          195479
si_culture_id     142317
brc_cat_id        117202
si_grouping_id     24298
biocyc_pgdb        17505
kegg_genome         6794
sg_assembly           10
Name: count, dtype: int64

In [4]:
vertices[vertices.vertex.str.contains("GCA_")].vertex_type.value_counts()

vertex_type
gb_assembly    275232
biocyc_pgdb         7
Name: count, dtype: int64

In [5]:
df = pd.read_parquet("../../preds1000/REL_output/preds_strainselect.parquet")

In [6]:
d = pd.read_parquet("../../preds920/REL_output/preds_strainselect.parquet")

In [7]:
df.StrainSelectID.unique().shape

(53870,)

In [8]:
d.StrainSelectID.unique().shape

(54579,)

In [9]:
d.vertex_type.value_counts()

vertex_type
gss               807168
brc_cat_id         11854
contig             10709
wgs_master_pre      2826
biocyc_pgdb         1277
si_culture_id        786
g16                  442
si_grouping_id       104
Name: count, dtype: int64

In [10]:
df.vertex_type.value_counts()

vertex_type
gss               763849
brc_cat_id         10021
contig              9266
wgs_master_pre      2110
biocyc_pgdb         1301
si_culture_id        975
g16                  727
si_grouping_id       187
kegg_genome           71
Name: count, dtype: int64

In [11]:
set(vertices.vertex_type.value_counts().index).difference(set(df.vertex_type.value_counts().index))

{'gb_assembly', 'gold_org', 'patric_genome', 'rs_assembly', 'sg_assembly'}

In [12]:
set(vertices.vertex_type.value_counts().index).difference(set(d.vertex_type.value_counts().index))

{'gb_assembly',
 'gold_org',
 'kegg_genome',
 'patric_genome',
 'rs_assembly',
 'sg_assembly'}

In [13]:
short_df = df[df.text.str.len() < 150]

In [16]:
re="STRAIN-COMPOUND:PRODUCES"

In [17]:
df[df.rel==re].sort_values("word_qc").to_csv(f"network_files/{re}.csv")